In [3]:
import re
import os
import time
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import matplotlib.pyplot as plt

# import tensorflow as tf
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential, Model, load_model
# from tensorflow.keras.layers import Embedding, LSTM, Dense, Flatten
# from tensorflow.keras.callbacks import EarlyStopping
# from keras_self_attention import SeqSelfAttention

# from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

# from underthesea import word_tokenize
# from collections import Counter
# import fasttext


#from google.colab import drive
#drive.mount('/content/gdrive')


#path = "/content/gdrive/MyDrive/KLTN/"


# Trích xuất thông tin

In [4]:
def extract_component(text, f_line):
    
    before_tag_match = []
    fist_line = []
    type_text = []
    rel_tag_match = []
    in_tag_val = []
    after_tag_val = []
    
    if not isinstance(text, str):
        text = str(text)

    match_find = re.findall(r'([^;.<()>:]*)<(\w+) rel="(\w+)">(.*?)</\2>([^;.<()>:]*)', text, re.DOTALL)
    if match_find and any(match for match in match_find if match):
        for match in match_find:
            before_tag_match.append(match[0].strip())
            fist_line.append(f_line)
            type_text.append(match[1])
            rel_tag_match.append(match[2])
            in_tag_val.append(match[3].strip())
            after_tag_val.append(match[4].strip())

    return before_tag_match, fist_line, type_text, rel_tag_match, in_tag_val, after_tag_val


# text = 'các nội_dung khác liên_quan đến việc thu , nộp , quản_lí , sử_dụng , chứng_từ thu , công_khai chế_độ thu phí sử_dụng đường_bộ không quy_định tại Nghị_định này , thực_hiện theo quy_định tại Luật phí và lệ_phí , <LT rel="DC"> Luật quản_lí thuế <NĐ rel="DC"> Nghị_định số 120/2016/nđ-cp </NĐ> </LT> , <NĐ rel="DC"> Nghị_định số 82/2023/nđ-cp </NĐ> , <NĐ rel="DC"> Nghị_định số 126/2020/nđ-cp ngày 19 tháng 10 năm 2020 </NĐ> của chính_phủ quy_định chi_tiết một_số điều của Luật quản_lí thuế , <NĐ rel="DC"> Nghị_định số 91/2022/nđ-cp ngày 30 tháng 10 năm 2022 </NĐ> của chính_phủ sửa_đổi , bổ_sung một_số điều của <NĐ rel="DC"> Nghị_định số 126/2020/nđ-cp ngày 19 tháng 10 năm 2020 </NĐ> của chính_phủ quy_định chi_tiết một_số điều của Luật quản_lí thuế ; <NĐ rel="DC"> Nghị_định số 123/2020/nđ-cp ngày 19 tháng 10 năm 2020 </NĐ> của chính_phủ quy_định về hoá_đơn , chứng_từ .'
# before_tag, _f, t_text, rel, in_tag, after_tag = extract_component(text, 'aaa')

# print("Rel:", rel)
# print("type:", _f)
# print("Type:", t_text)
# print("Trước thẻ:", before_tag)
# print("Trong thẻ:", in_tag)
# print("Sau thẻ:", after_tag)

In [ ]:
# Đường dẫn tới thư mục chứa các file XML
folder_path = "../data/data_label/"
file_list = os.listdir(folder_path)

_type, first_line, rel, before_tag, in_tag, after_tag = [], [], [], [], [], []

# Loop through each XML file
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        f_line = file.readline().strip()
        file_child = file.read().splitlines()[1:]
        for text in file_child:
            b_tag, list_f_line, t_text, _rel, i_tag, af_tag = extract_component(text, f_line)
            if i_tag:
                _type.extend(t_text)
                first_line.extend(list_f_line)
                rel.extend(_rel)
                before_tag.extend(b_tag)
                in_tag.extend(i_tag)
                after_tag.extend(af_tag)
            else:
                continue


data = {
    'Loại': _type,
    'Thực thể tham chiếu': first_line,
    'Quan hệ': rel,
    'Trước': before_tag,
    'Thực thể được tham chiếu': in_tag,
    'Sau': after_tag
}
df = pd.DataFrame(data) 

df.astype(str).to_csv('../data/tvpl_data_model.csv', index=False, encoding="utf-8-sig")

In [9]:
df.astype(str).to_csv('../data/tvpl_data_model.csv', index=False, encoding="utf-8-sig")

# Visualize

In [15]:
dt = pd.read_csv("data/vbpl_data.csv", encoding="utf-8")

A = "Thực thể tham chiếu"
Label = "Quan hệ"
B = "Thực thể được tham chiếu"
Before_B = "Trước"
After_B = "Sau"

data_Clean = dt[[A,Label,Before_B,B,After_B]].copy()
data_Clean[Label] = dt[Label].fillna('None')
data_Clean[[A, B]].dropna()

print(data_Clean[Label].unique())
data_Clean.info()
data_Clean.head()



['CC' 'DC' 'DSD' 'BTT' 'HHL' 'None' 'DHD']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97626 entries, 0 to 97625
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Thực thể tham chiếu       97626 non-null  object
 1   Quan hệ                   97626 non-null  object
 2   Trước                     77294 non-null  object
 3   Thực thể được tham chiếu  97626 non-null  object
 4   Sau                       69685 non-null  object
dtypes: object(5)
memory usage: 3.7+ MB


,Thực thể tham chiếu,Quan hệ,Trước,Thực thể được tham chiếu,Sau
0,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,CC,căn_cứ,Hiến_pháp nước cộng_hoà xã_hội chủ_nghĩa việt_nam,NaN
1,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC,"trường_hợp cơ_quan , đơn_vị , tổ_chức thực_hiệ...",Luật quản_lí thuế_tính trên số,tiền chậm trả và thời_gian chậm trả
2,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC,một trong các giấy_tờ được lập trong quá_trình...,Chỉ_thị số 299/tg ngày 10 tháng 11 năm 1980,"của thủ_tướng chính_phủ về công_tác đo_đạc , p..."
3,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC,giấy_tờ của đơn_vị quốc_phòng giao đất cho cán...,Chỉ_thị số 282/ct-qp ngày 11 tháng 7 năm 1991,của bộ_trưởng bộ quốc_phòng mà việc giao đất đ...
4,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC,hợp_đồng mua_bán nhà ở thuộc sở_hữu nhà_nước t...,Nghị_định số 61-cp ngày 05 tháng 7 năm 1994,của chính_phủ về mua_bán và kinh_doanh nhà ở h...


In [18]:
groups = data_Clean.groupby(by=Label).count()[A]
fig = go.Figure()
fig.add_trace(go.Bar(
    x=groups.index,
    y=groups,
    marker_color='cadetblue',
    width=[0.4, 0.4]))
fig.update_layout(title='Classes and their number in the dataset', title_x=0.5)

In [23]:
dt_clean = data_Clean.copy()
dt_clean['Text'] = dt_clean.apply(lambda d: f'{d[A]} {d[Before_B]} {d[B]} {d[After_B]}'.strip(), axis=1)

#dt = dt.drop(['Thực thể tham chiếu', 'Trước', 'Thực thể được tham chiếu', 'Sau'], axis=1)
#dt.to_csv('output1.csv', index=False, encoding="utf-8-sig")
dt_clean.head()

,Thực thể tham chiếu,Quan hệ,Trước,Thực thể được tham chiếu,Sau,Text
0,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,CC,căn_cứ,Hiến_pháp nước cộng_hoà xã_hội chủ_nghĩa việt_nam,NaN,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...
1,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC,"trường_hợp cơ_quan , đơn_vị , tổ_chức thực_hiệ...",Luật quản_lí thuế_tính trên số,tiền chậm trả và thời_gian chậm trả,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...
2,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC,một trong các giấy_tờ được lập trong quá_trình...,Chỉ_thị số 299/tg ngày 10 tháng 11 năm 1980,"của thủ_tướng chính_phủ về công_tác đo_đạc , p...",Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...
3,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC,giấy_tờ của đơn_vị quốc_phòng giao đất cho cán...,Chỉ_thị số 282/ct-qp ngày 11 tháng 7 năm 1991,của bộ_trưởng bộ quốc_phòng mà việc giao đất đ...,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...
4,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC,hợp_đồng mua_bán nhà ở thuộc sở_hữu nhà_nước t...,Nghị_định số 61-cp ngày 05 tháng 7 năm 1994,của chính_phủ về mua_bán và kinh_doanh nhà ở h...,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...


In [24]:
def check_length(data):
    min_length = float('inf')
    max_length = 0
    total_length = 0
    count = 0
    length_counts = {}

    for text in data:
        length = len(text)

        # Update minimum length
        if length < min_length:
            min_length = length
            # m = count

        # Update maximum length
        if length > max_length:
            max_length = length

        # Update total length for average calculation
        total_length += length

        # Update length counts for mode calculation
        if length in length_counts:
            length_counts[length] += 1
        else:
            length_counts[length] = 1

        count += 1

    average_length = round(total_length / count)
    mode_length = max(length_counts, key=length_counts.get)
    print("Minimum length:", min_length)
    print("Maximum length:", max_length)
    print("Average length:", average_length)
    print("Mode length:", mode_length)

dt_preprocess =  dt_clean[['Text', 'Quan hệ']].copy()
dt_preprocess.head()


,Text,Quan hệ
0,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,CC
1,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC
2,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC
3,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC
4,Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024...,DC


In [25]:
check_length(dt_preprocess['Text'])


Minimum length: 83
Maximum length: 1115
Average length: 234
Mode length: 126


# Lựa chọn tham số

In [ ]:
max_length = 280  # Chiều dài tối đa của câu lấy theo độ dài trung bình của toàn bộ tập dữ liệu
vocab_size = 10000  # Kích thước từ vựng
embedding_dim = 100  # Kích thước nhúng
lstm_units = 128  # Số đơn vị LSTM

Tokenize

In [ ]:
def tokenization(data):
    Text_col = data
    tokenizer = Tokenizer(split=' ')
    tokenizer.fit_on_texts(Text_col)
    words_to_index = tokenizer.word_index
    vocab_size = len(words_to_index) + 1

    # chuyển đổi dữ liệu văn bản thành các chuỗi số
    tokenizer_seq = tokenizer.texts_to_sequences(Text_col)

    # Đảm bảo mỗi sequece có cùng độ dài
    # 'post' có nghĩa là thêm padding vào cuối mỗi sequence
    # 'pre' có nghĩa là thêm padding vào đầu mỗi sequence
    tokenizer_pad = pad_sequences(tokenizer_seq, padding='post', maxlen=max_length)

    return tokenizer_pad, vocab_size, words_to_index

In [ ]:
# Tách từ
data_tokenizer = dt_preprocess.copy()
data_tokenizer['Text'] = data_tokenizer['Text'].apply(word_tokenize)

data_tokenizer.info()
data_tokenizer.head()


data_tokenizer_pad, vocab_size, words_to_index = tokenization(data_tokenizer['Text'])

Create a Embedding model

In [ ]:

with open('data/fasttext/text_vbpl.txt', 'w', encoding="utf-8") as file:
    # Write content to the
    for i in dt_preprocess['Text']:
        file.write(i+"\n")

In [ ]:
model = fasttext.train_unsupervised('data/fasttext/text_vbpl.txt', maxn=0, dim=embedding_dim)
model.save_model('data/fasttext/vbpl_fasttext_{embedding_dim}d.bin')

In [ ]:


all_text = ' '.join(dt_preprocess['Text'])

# Split the text into individual words
words = all_text.split()

# Count the frequency of each word
word_counts = Counter(words)

# Find the word with the highest frequency
most_common_word, frequency = word_counts.most_common(1)[0]

print("Most common word:", most_common_word)
print("Frequency:", frequency)

In [ ]:

ft = fasttext.load_model('data/fasttext/vbpl_fasttext_100d.bin')
embed_vector_size = ft.get_word_vector('của').shape[0]

embed_vector_matrix = np.zeros((vocab_size, embed_vector_size))

for word, i in words_to_index.items():
    embedding_vector = ft.get_word_vector(word)
    if any(embedding_vector):
        embed_vector_matrix[i] = embedding_vector
    else:
        embed_vector_matrix[i] = np.random.uniform(low=-1, high=1, size= embed_vector_size)

print(f"Max length | input_length      = {max_length}")
print(f"input_dim  | vocab_size        = {vocab_size}")
print(f"output_dim | embed_vector_size = {embed_vector_size}")
print(f"weights = {embed_vector_matrix.shape}")

embedding = Embedding(input_dim=vocab_size, output_dim=embed_vector_size,
                      input_length=max_length, weights=[embed_vector_matrix], trainable=False)

Model

In [ ]:
classes = data_tokenizer['Quan hệ'].nunique()
print(f"Number labels = {classes}")

In [ ]:
def create_model():
    model = Sequential()
    model.add(embedding)
    model.add(LSTM(lstm_units, return_sequences=True, dropout=0.4,recurrent_dropout=0.2))
    model.add(LSTM(lstm_units, return_sequences=True, dropout=0.4,recurrent_dropout=0.2))
    model.add(LSTM(lstm_units, return_sequences=True, dropout=0.4,recurrent_dropout=0.2))
    model.add(SeqSelfAttention())
    model.add(Flatten())
    model.add(Dense(classes, activation="softmax"))

    # Model compiling
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# Chia dataset thành tập training thành train_val và test
X = data_tokenizer_pad
y = data_tokenizer['Quan hệ']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y.values, test_size=0.2, random_state=40)

In [ ]:
# encode label
label_encoder = LabelEncoder()
label_encoder.fit(data_tokenizer["Quan hệ"].values)
print('\nLabels:',list(label_encoder.classes_), '\n')
y_train_val = label_encoder.transform(y_train_val)
y_test = label_encoder.transform(y_test)

# Chia train_val thành train và validation
X_train, X_val, y_train, y_val  = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=40)

print("Size Train: ",X_train.shape, y_train.shape)
print("Size Test: ",X_test.shape, y_test.shape)
print("Size Validation: ",X_val.shape, y_val.shape)

Run


In [ ]:
model = create_model()
batchSize = round(X_train.shape[0] * 0.075)
epoch = 5
print(batchSize)
model.summary()


In [ ]:
# Ghi lại thời điểm bắt đầu
start_time = time.time()
checkpoint_path = "".join(['checkpoint','vbpl_ft',"/cp.ckpt"])
checkpoint_dir = os.path.dirname(checkpoint_path)
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
checkpointing = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
lstm_model_history = model.fit(X_train, y_train, batch_size=batchSize, epochs=epoch, callbacks=[earlystopping,checkpointing], validation_data=(X_val, y_val))

# Tính thời gian đã trôi qua
training_time = round(time.time() - start_time, 2)

In [ ]:
path = f"model/classify_law_rel_fasttext_{max_length}_{batchSize}_{epoch}_{X_train.shape[0]}_{training_time}.h5"
print(path)
model.save(path)

In [ ]:
# Predictions on the Test Set
score = model.evaluate(X_test, y_test)

# Model Performance
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
def plot_graph(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.figure(figsize=(10,6))
    plt.plot(epochs, acc, 'r', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
    plt.plot(epochs, loss, 'g', label='Training Loss')
    plt.plot(epochs, val_loss, 'y', label='Validation Loss')

    plt.title(f'{X_train.shape[0]} Data - {batchSize} batch size - {epoch} epochs')
    plt.legend(loc=0)
    plt.savefig(f'images/law_fasttext_{max_length}_{X_train.shape[0]}_{batchSize}_{epoch}_{training_time}.png')
    plt.show()


plot_graph(lstm_model_history)

In [ ]:
start = None
end = None

# Phân loại 
y_pred = model.predict(X_test[start:end])
# Mã hóa label
y_pred_original = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))
y_test_original = label_encoder.inverse_transform(y_test[start:end])

# In ra kết quả phân loại
print(f"Kết quả thực tế: {y_test_original}")
print(f"Kết quả mô hình phân loại: {y_pred_original} - {y_pred_original == y_test_original}")

In [ ]:
# Tìm tất cả các nhãn duy nhất
labels = np.unique(np.concatenate((y_test_original, y_pred_original)))

# Tạo ma trận confusion
confusion = confusion_matrix(y_test_original, y_pred_original, labels=labels)

print("Ma trận confusion:")
print(confusion)

In [ ]:
# Calculate precision, recall, and F1 score for each class
precision_per_class = precision_score(y_test_original, y_pred_original, average=None)
recall_per_class = recall_score(y_test_original, y_pred_original, average=None)
f1_per_class = f1_score(y_test_original, y_pred_original, average=None)

# Calculate macro-average precision, recall, and F1 score
macro_precision = round(np.mean(precision_per_class),3)
macro_recall = round(np.mean(recall_per_class),3)
macro_f1 = round(np.mean(f1_per_class),3)

# Create a DataFrame for the table
metrics_table_per_class = pd.DataFrame({
    'Class': np.unique(y_test_original),
    'Precision': precision_per_class,
    'Recall': recall_per_class,
    'F1 Score': f1_per_class
})

# Print the table per class
print("Metrics Table per Class:")
print(metrics_table_per_class)

# Print macro-average metrics
print("\nMacro-Average Metrics:")
print(f"Macro Precision: {macro_precision}")
print(f"Macro Recall: {macro_recall}")
print(f"Macro F1 Score: {macro_f1}")